In [33]:
def debug(m: Array[Array[Int]]) = {
    (0 to m(0).length-1) foreach { j =>
        print(s"$j,")
    }
    println("________________________")
    for { row <- m } {
        for { e <- row } {
            print(s"$e,")
        }
        println("________________")
    }
}

def knapsack(maxWeight: Int, value: Vector[Int], weight: Vector[Int]): Int = {
    val n = value.length
    //println(s"n=$n")
    val solutions: Array[Array[Int]] = Array.fill(n+1, maxWeight + 1)( 0 )
    (1 to n) foreach { i =>
        (1 to maxWeight) foreach { j =>
            solutions(i)(j) = if( j - weight(i-1) >= 0 ) {
                Math.max( solutions(i-1)(j) , solutions(i-1)(j - weight(i-1)) + value(i-1) )
            } else {
                solutions(i-1)(j)
            }
        }
    } 
    //debug(solutions)
    solutions(n)(maxWeight)
}

object Tester {

    case class TestCase(maxWeight: Int, values: Vector[Int], weights: Vector[Int], expectedItems: Vector[Int])

    def testKnapsack(f: (Int, Vector[Int], Vector[Int]) => Int)(testCase: TestCase) = {
        val TestCase(maxWeight,values,weights,expectedItems) = testCase
        val expectedProfit = (values zip expectedItems).map { case (a,b) => a*b }.sum
        val returned = f(maxWeight, values, weights)
        assert(returned == expectedProfit, s"Expected $expectedProfit got $returned")
        println("Test Passed!")
    }


    val testCase1 = TestCase(
      maxWeight = 165,
      values = Vector(92, 57, 49, 68, 60, 43, 67, 84, 87, 72),
      weights = Vector(23, 31, 29, 44, 53, 38, 63, 85, 89, 82),
      expectedItems = Vector(1, 1,1,1,0,1,0,0,0,0)
    )

    val testCase2 = TestCase(
      maxWeight = 26,
      values = Vector(24,13,23,15,16),
      weights = Vector(12,7,11,8,9),
      expectedItems = Vector(0,1,1,1,0)
    )

    val testCase3 = TestCase(
      maxWeight = 190,
      values = Vector(50,50,64,46,50,5),
      weights = Vector(56,59,80,64,75,17),
      expectedItems = Vector(1,1,0,0,1,0)
    )

    val testCase4 = TestCase(
      maxWeight = 50,
      values = Vector(70,20,39,37,7,5,10),
      weights = Vector(31,10,20,19,4,3,6),
      expectedItems = Vector(1,0,0,1,0,0,0)
    )

    val testCase5 = TestCase(
      maxWeight = 104,
      values = Vector(350,400,450,20,70,8,5,5),
      weights = Vector(25,35,45,5,25,3,2,2),
      expectedItems = Vector(1,0,1,1,1,0,1,1)
    )

    val testCase6 = TestCase(
      maxWeight = 170,
      values = Vector(442,525,511,593,546,564,617),
      weights = Vector(41,50,49,59,55,57,60),
      expectedItems = Vector(0,1,0,1,0,0,1)
    )

    def test(f: (Int, Vector[Int], Vector[Int]) => Int) = {
        List(
            testCase1,
            testCase2,
            testCase3,
            testCase4,
            testCase5,
            testCase6
        ) foreach testKnapsack(f)
    }
}

Tester.test(knapsack)

Test Passed!
Test Passed!
Test Passed!
Test Passed!
Test Passed!
Test Passed!


defined function debug
defined function knapsack
defined object Tester

In [35]:
def knapsack(maxWeight: Int, value: Vector[Int], weight: Vector[Int]): Int = {
    val n = value.length
    var solutions: Array[Int] = Array.fill(maxWeight + 1)( 0 )
    (1 to n) foreach { i =>
        val newSolutions = Array.fill(maxWeight + 1)( 0 )
        (1 to maxWeight) foreach { j =>
            newSolutions(j) = if( j - weight(i-1) >= 0 ) {
                Math.max( solutions(j) , solutions(j - weight(i-1)) + value(i-1) )
            } else {
                solutions(j)
            }
        }
        solutions = newSolutions
    } 
    solutions(maxWeight)
}

Tester.test(knapsack)

Test Passed!
Test Passed!
Test Passed!
Test Passed!
Test Passed!
Test Passed!


defined function knapsack

In [1]:
case class State[A,S](run: S => (A,S)) extends AnyRef { self =>
    
    def map[B](f: A => B): State[B,S] = State { st =>
        val (a,st2) = self.run(st)
        (f(a), st2)
    }
                                                       
                                                       
    def flatMap[B](f: A => State[B,S]): State[B,S] = State { st =>
        val (a,st2) = self.run(st)
        f(a).run(st2)
    }
                                                    
    
}

object State {
    
    def succeed[S,A](a: A): State[A, S] = State( st => (a, st) )
    
    def modify[S](f: S => S): State[Unit, S] = State( st => ( (), f(st) ))
    
    def set[S](st: S): State[Unit, S] = modify(_ => st)
    
    def get[S](): State[S,S] = State( st => (st,st) )
    
    def traverse_[A,B,S](as: Iterable[A])(f: A => State[B,S]): State[Unit,S] = {
        as.foldLeft( succeed[S,Unit]( () ) ) { (st, a) =>
            for {
                _ <- st
                _ <- f(a)
            } yield ()
        }
    }
    
    def traverse[A,B,S](as: Vector[A])(f: A => State[B,S]): State[Vector[B],S] = {
        as.foldRight( succeed[S,Vector[B]](Vector.empty) ) { (a,stv) =>
            for {
                head <- f(a)
                tail <- stv
            } yield head +: tail
        }
    }
    
}

defined class State
defined object State

In [37]:
type SolutionsState = State[Unit,Vector[Vector[Int]]]

def modifySolution(i: Int, j: Int, newVal: Int)(solutions: Vector[Vector[Int]]): Vector[Vector[Int]] = {
    solutions.updated(i, solutions(i).updated(j, newVal))
}

def knapsack(maxWeight: Int, value: Vector[Int], weight: Vector[Int]): Int = {
    val n = value.length
    val initialState: Vector[Vector[Int]] = Vector.fill(n+1, maxWeight + 1)( 0 )
    val st: SolutionsState = State.traverse_( 1 to n ) { i =>
        State.traverse_( 1 to maxWeight ) { j =>
            for {
                solutions <- State.get[Vector[Vector[Int]]]()
                newVal = if( j - weight(i-1) >= 0 ) {
                    Math.max( solutions(i-1)(j) , solutions(i-1)(j - weight(i-1)) + value(i-1) )
                } else {
                    solutions(i-1)(j)
                }
                _ <- State.modify(modifySolution(i,j,newVal))
            } yield ()
        }
    }
    val (_,solution) = st.run(initialState)
    solution(n)(maxWeight)
}

Tester.test(knapsack)

Test Passed!
Test Passed!
Test Passed!
Test Passed!
Test Passed!
Test Passed!


defined type SolutionsState
defined function modifySolution
defined function knapsack

In [38]:
type SolutionsState = State[Unit,Vector[Int]]

def modifySolution(j: Int, newVal: Int)(solutions: Vector[Int]): Vector[Int] = {
    solutions.updated(j, newVal)
}

def knapsack(maxWeight: Int, value: Vector[Int], weight: Vector[Int]): Int = {
    val n = value.length
    val initialState: Vector[Int] = Vector.fill(maxWeight + 1)( 0 )
    val st: SolutionsState = State.traverse_( 1 to n ) { i =>
        for {
            solutions <- State.get[Vector[Int]]()
            newSolutions = 0 +: ( 1 to maxWeight ).map { j: Int =>
                if( j - weight(i-1) >= 0 ) {
                    Math.max( solutions(j) , solutions(j - weight(i-1)) + value(i-1) )
                } else {
                    solutions(j)
                }
            }.toVector
            _ <- State.set(newSolutions)
        } yield ()
    }
    val (_,solution) = st.run(initialState)
    solution(maxWeight)
}

Tester.test(knapsack)

Test Passed!
Test Passed!
Test Passed!
Test Passed!
Test Passed!
Test Passed!


defined type SolutionsState
defined function modifySolution
defined function knapsack